#### Data Collecting and Pre-Processing
##### Crawling data from giphy via their API. 
##### Convert .gif image to VGG final dense layer


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import matplotlib.image as Image
import keras
import json,urllib,os,sys
%matplotlib inline
from scipy.misc import imresize
from datetime import datetime, timedelta
import h5py as h5py
try:
    utils
except:
    import utils
else:    
    reload(utils)
    print 'utils has been reloaded'

In [ ]:
# EMBEDDING CLASS TO BE USED
class Embedding:
    def __init__(self, topology='vgg16'):
        self.topology = topology
        self.network = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet',
                                                      input_tensor=None, input_shape=None,
                                                      pooling=None, classes=1000)
        self.network.layers.pop() # Get rid of the classification layer
        self.network.outputs = [self.network.layers[-1].output]
        self.network.layers[-1].outbound_nodes = []
        for layer in self.network.layers: layer.trainable=False
    def get_vector(self, batch):
        return self.network.predict(batch)

In [ ]:
API_KEY = 'J9rmStnqj0ZhQFObVxh81O84hb7UQZjY'
N_FILES = 100
SEARCH_TERM = 'human'
api_path = "http://api.giphy.com/v1/gifs/search?q={}&api_key={}&limit={}".format(SEARCH_TERM,
                                                                                 API_KEY,
                                                                                 N_FILES)
data = json.loads(urllib.urlopen(api_path).read())
utils.save_files(data, SEARCH_TERM)

In [ ]:
embeddings = Embedding()

In [ ]:
# Pre-process gif to series of images which can be 
# fed to netwok.
files = os.listdir('gifs')
seq_length = 11 # 10 for training, 1 for the target
for idx, gif_file in enumerate(files):
    if '.gif' in gif_file:
        gif_file_path  = 'gifs/'+gif_file
        frames = utils.processImage(gif_file_path,
                                    reshape_to_vgg=True,
                                    image_limit=20)
        if not frames.shape[0] < 11:
            embedding_matrix = embeddings.get_vector(frames)
            max_index = embedding_matrix.shape[0]
            data_arr = []
            for i in range(max_index - seq_length + 1):
                start_ix = i
                end_ix = i+seq_length
                target_ix = end_ix
                data_arr.append(embedding_matrix[start_ix:end_ix,:])
            data_set = utils.Dataset(root_dir='data/gifs_300')
            data_set.add(data_arr)
            print 'Processed %i of %i'%(idx,len(files))
        else:
            print '%s has less than %i frames, skipping.'%(gif_file,seq_length)
        # pyplot.imshow(frames[0])